<br>

# Técnicas Matemáticas para Big Data - Project NN?
<br><br>


GROUP NN:
- Rodrigo Pinto - Nº 103280 - ??% Work Participation
- Student 2 - Nº xxxxx - ??% Work Participation

<br><br>

## 1. Introduction to the problem of study [1,0 valor]

<br><br>
## 2. Brief and general description of the approach and methods used [1,5 valor]

<br><br>
## 3. Brief History and literature review of the problem and methods/algorithms [1,5 valor]

<br><br>
## 4. About the main method/algorithm used [1,5 valor]

<br><br>

## 5. Python imports and global configurations [0,5 valor]

### Install and import the necessary libraries to compute the Bayesian Network and perform other methods  

In [2]:
# %pip install pandas
# %pip install seaborn
# %pip install matplotlib
# %pip install numpy
# %pip install pomegranate
# %pip install torch
# %pip install Pillow
%pip install duckdb pandas requests tqdm


   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/12.3 MB 9.5 MB/s eta 0:00:02
   ----------- ---------------------------- 3.4/12.3 MB 10.1 MB/s eta 0:00:01
   ---------------- ----------------------- 5.2/12.3 MB 9.7 MB/s eta 0:00:01
   ------------------------ --------------- 7.6/12.3 MB 9.8 MB/s eta 0:00:01
   ------------------------------- -------- 9.7/12.3 MB 9.9 MB/s eta 0:00:01
   -------------------------------------- - 11.8/12.3 MB 9.7 MB/s eta 0:00:01
   ---------------------------------------- 12.3/12.3 MB 9.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


<br><br>

## 6. Dataset and variables explanation [1,5 valor]

In [4]:
import os
import gzip
import json
import requests
import duckdb
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm

# ----------------------------------------------
# CONFIGURAÇÕES
# ----------------------------------------------
# Pasta local onde os ficheiros .json.gz serão guardados
DATA_DIR = "gharchive_data"
os.makedirs(DATA_DIR, exist_ok=True)

# Base de dados DuckDB
DB_PATH = "github_archive.duckdb"

# Intervalo de tempo (exemplo: 3 dias)
START_DATE = datetime(2024, 10, 1)
END_DATE = datetime(2024, 10, 2)  # até, mas não incluindo este dia

# ----------------------------------------------
# FUNÇÃO PARA DOWNLOAD E LEITURA DE UM FICHEIRO
# ----------------------------------------------
def download_and_parse(date_obj, hour):
    """Faz download de um ficheiro GHArchive para uma data/hora específica."""
    date_str = date_obj.strftime("%Y-%m-%d")
    url = f"https://data.gharchive.org/{date_str}-{hour}.json.gz"
    local_path = os.path.join(DATA_DIR, f"{date_str}-{hour}.json.gz")

    # Se já existir localmente, não faz download novamente
    if not os.path.exists(local_path):
        r = requests.get(url, timeout=60)
        if r.status_code == 200:
            with open(local_path, "wb") as f:
                f.write(r.content)
        else:
            print(f"[Aviso] Falhou o download de {url} (status {r.status_code})")
            return []

    # Lê o conteúdo
    with gzip.open(local_path, "rb") as f:
        data = f.read().decode("utf-8").splitlines()

    # Extrai apenas os campos relevantes
    records = []
    for line in data:
        try:
            e = json.loads(line)
            records.append({
                "actor_id": e.get("actor", {}).get("id"),
                "actor_login": e.get("actor", {}).get("login"),
                "repo_name": e.get("repo", {}).get("name"),
                "type": e.get("type"),
                "created_at": e.get("created_at")
            })
        except json.JSONDecodeError:
            continue

    return records

# ----------------------------------------------
# LOOP PRINCIPAL DE DOWNLOAD + ARMAZENAMENTO
# ----------------------------------------------
all_records = []

current_date = START_DATE
while current_date < END_DATE:
    for hour in tqdm(range(24), desc=f"Processando {current_date.date()}"):
        records = download_and_parse(current_date, hour)
        all_records.extend(records)
    current_date += timedelta(days=1)

# Converte para DataFrame
df = pd.DataFrame(all_records)
print(f"\nTotal de eventos recolhidos: {len(df):,}")

# ----------------------------------------------
# GRAVAÇÃO NA BASE DE DADOS DUCKDB
# ----------------------------------------------
con = duckdb.connect(DB_PATH)
con.execute("""
CREATE TABLE IF NOT EXISTS github_events (
    actor_id BIGINT,
    actor_login VARCHAR,
    repo_name VARCHAR,
    type VARCHAR,
    created_at TIMESTAMP
)
""")

con.register("tmp_df", df)
con.execute("INSERT INTO github_events SELECT * FROM tmp_df")
con.close()

print(f"\n✅ Dados guardados com sucesso em {DB_PATH}")


Processando 2024-10-01:  29%|██▉       | 7/24 [01:19<03:11, 11.29s/it]


KeyboardInterrupt: 

In [5]:
# Caminhos
DATA_DIR = "gharchive_data"
DB_PATH = "github_archive.duckdb"

# Função para ler ficheiro local
def read_local_file(path):
    """Lê ficheiro JSON.GZ local e devolve lista de registos relevantes."""
    with gzip.open(path, "rb") as f:
        data = f.read().decode("utf-8").splitlines()

    records = []
    for line in data:
        try:
            e = json.loads(line)
            records.append({
                "actor_id": e.get("actor", {}).get("id"),
                "actor_login": e.get("actor", {}).get("login"),
                "repo_name": e.get("repo", {}).get("name"),
                "type": e.get("type"),
                "created_at": e.get("created_at")
            })
        except json.JSONDecodeError:
            continue
    return records


# Lê todos os ficheiros .json.gz da pasta
files = sorted([f for f in os.listdir(DATA_DIR) if f.endswith(".json.gz")])

all_records = []
for file in tqdm(files, desc="A carregar ficheiros locais"):
    path = os.path.join(DATA_DIR, file)
    records = read_local_file(path)
    all_records.extend(records)

# Converte para DataFrame
df = pd.DataFrame(all_records)
print(f"\nTotal de eventos extraídos: {len(df):,}")

# Conexão à base DuckDB
con = duckdb.connect(DB_PATH)

# Cria tabela se não existir
con.execute("""
CREATE TABLE IF NOT EXISTS github_events (
    actor_id BIGINT,
    actor_login VARCHAR,
    repo_name VARCHAR,
    type VARCHAR,
    created_at TIMESTAMP
)
""")

# Regista dataframe temporário e insere na tabela
con.register("tmp_df", df)
con.execute("INSERT INTO github_events SELECT * FROM tmp_df")
con.close()

print(f"✅ Inseridos {len(df):,} novos registos na base {DB_PATH}")

A carregar ficheiros locais: 100%|██████████| 48/48 [10:45<00:00, 13.44s/it]



Total de eventos extraídos: 10,174,100
✅ Inseridos 10,174,100 novos registos na base github_archive.duckdb


In [2]:
import duckdb
con = duckdb.connect("github_archive.duckdb")

df = con.execute("""
SELECT actor_id, actor_login, repo_name, type, created_at
FROM github_events
ORDER BY created_at DESC
LIMIT 20
""").fetchdf()

df

,actor_id,actor_login,repo_name,type,created_at
0,34755028,ivanagas,PostHog/posthog.com,PullRequestEvent,2024-10-02 23:59:59
1,41898282,github-actions[bot],ePlus-DEV/metrics,PushEvent,2024-10-02 23:59:59
2,165398914,net7toulouse,inp-net/churros,PushEvent,2024-10-02 23:59:59
3,171880006,Peterparkerbr23,Peterparkerbr23/pedro3f,CreateEvent,2024-10-02 23:59:59
4,113317341,prkr9,prkr9/sshl,PushEvent,2024-10-02 23:59:59
5,177841759,correa-augusto,correa-augusto/primeiroRepositorioWeb,DeleteEvent,2024-10-02 23:59:59
6,174059141,HoucineMohamed,HoucineMohamed/HoucineMohamed,CreateEvent,2024-10-02 23:59:59
7,116701466,hashsllngingslasher,hashsllngingslasher/bigdata-jupytor,PushEvent,2024-10-02 23:59:59
8,128195885,JaydenCruz2004,JaydenCruz2004/JaydenCruz2004.github.io,CreateEvent,2024-10-02 23:59:59
9,9019229,sayboras,cilium/cilium,PullRequestEvent,2024-10-02 23:59:59


In [5]:

a = con.execute("""SELECT COUNT(*) AS total_eventos
FROM github_events;""").fetchone()[0]
a



10174100

In [6]:

con = duckdb.connect("github_archive.duckdb")

schema = con.execute("DESCRIBE github_events").fetchdf()
schema


,column_name,column_type,null,key,default,extra
0,actor_id,BIGINT,YES,None,None,None
1,actor_login,VARCHAR,YES,None,None,None
2,repo_name,VARCHAR,YES,None,None,None
3,type,VARCHAR,YES,None,None,None
4,created_at,TIMESTAMP,YES,None,None,None


<br><br>

## 7. Main code as possible solution to the problem [1,5 valor] 

<br><br>

## 8. Analysis of Example 1 [3,0 valor]

<br><br>

## 9. Analysis of Example 2 [3,0 valor]

<br><br>
## 10. Pros and cons of the approach [2,0 valor]

<br><br>
## 11. Future improvements [2,0 valor]

<br>
<div style="text-align: center;">
    <br><br>
    <p style="font-size: 40px;">References [1,0 valor]</p>
</div>
<br>
